In [7]:
import pandas as pd

# Load the datasets
df_customers = pd.read_csv('E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Customers.csv', encoding='latin-1')
df_exc_rates = pd.read_csv('E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Exchange_rates.csv', encoding='latin-1')
df_products = pd.read_csv('E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Products.csv', encoding='latin-1')
df_sales = pd.read_csv('E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Sales.csv', encoding='latin-1')
df_stores = pd.read_csv('E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Stores.csv', encoding='latin-1')

# Rename columns to replace spaces with underscores
for df in [df_customers, df_exc_rates, df_products, df_sales, df_stores]:
    df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Dropping specified columns from customers DataFrame
df_customers.drop(columns=['Name', 'State_Code', 'Zip_Code'], inplace=True)

# Dropping specified columns from products DataFrame
df_products.drop(columns=['Color', 'Subcategory', 'Category'], inplace=True)

# Convert date columns to datetime
df_exc_rates['Date'] = pd.to_datetime(df_exc_rates['Date'], format='%d/%m/%Y', errors='coerce')
df_customers['Birthday'] = pd.to_datetime(df_customers['Birthday'], format='%d/%m/%Y', errors='coerce')
df_sales['Order_Date'] = pd.to_datetime(df_sales['Order_Date'], format='%d/%m/%Y', errors='coerce')
df_stores['Open_Date'] = pd.to_datetime(df_stores['Open_Date'], format='%d/%m/%Y', errors='coerce')
df_sales.drop(columns=['Delivery_Date'], inplace=True)  # Drop Delivery_Date column if not needed

# Adjust data types
df_sales['Order_Number'] = df_sales['Order_Number'].astype('int64')
df_sales['Line_Item'] = df_sales['Line_Item'].astype('int8')
df_sales['CustomerKey'] = df_sales['CustomerKey'].astype('int64')
df_sales['StoreKey'] = df_sales['StoreKey'].astype('int8')
df_sales['ProductKey'] = df_sales['ProductKey'].astype('int16')
df_sales['Quantity'] = df_sales['Quantity'].astype('int8')
df_products['SubcategoryKey'] = df_products['SubcategoryKey'].astype('int16')
df_products['CategoryKey'] = df_products['CategoryKey'].astype('int8')
df_products['Unit_Cost_USD'] = df_products['Unit_Cost_USD'].replace('[\$,]', '', regex=True).astype(float)
df_products['Unit_Price_USD'] = df_products['Unit_Price_USD'].replace('[\$,]', '', regex=True).astype(float)

# Handle missing values
def handle_missing_values(df):
    # Identify columns with more than 80% missing values
    high_null_columns = [col for col in df.columns if (df[col].isnull().sum() / len(df)) * 100 > 78]
    # Drop columns with more than 80% missing values
    df.drop(columns=high_null_columns, inplace=True)
    
    for col in df.columns:
        # Handle missing values for numerical columns
        if df[col].dtype in ["int64", "float64", "int8", "int16"]:
            skewness = df[col].skew()
            if -0.4 <= skewness <= 0.4:
                df[col].fillna(df[col].mean(), inplace=True)
            else:
                df[col].fillna(df[col].median(), inplace=True)
        # Handle missing values for categorical columns
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    
    return df

# Apply the function to each DataFrame
datasets = {
    'Customers': df_customers,
    'Exchange_Rates': df_exc_rates,
    'Products': df_products,
    'Sales': df_sales,
    'Stores': df_stores
}

for name, df in datasets.items():
    datasets[name] = handle_missing_values(df)
    # Print out any remaining missing values
    print(f"{name} missing values:\n", datasets[name].isnull().sum())
    # Convert to CSV and save
    output_path = f'E:\\GUVI\\PROJECT 2 DATASPARK\\datasets\\Processed_{name}.csv'
    datasets[name].to_csv(output_path, index=False)

# Print data types after processing
print("\nData types after processing:")
for name, df in datasets.items():
    print(f"{name}:\n", df.dtypes)

print("Missing values handled and CSV files saved.")


Customers missing values:
 CustomerKey    0
Gender         0
City           0
State          0
Country        0
Continent      0
Birthday       0
dtype: int64
Exchange_Rates missing values:
 Date        0
Currency    0
Exchange    0
dtype: int64
Products missing values:
 ProductKey        0
Product_Name      0
Brand             0
Unit_Cost_USD     0
Unit_Price_USD    0
SubcategoryKey    0
CategoryKey       0
dtype: int64
Sales missing values:
 Order_Number     0
Line_Item        0
Order_Date       0
CustomerKey      0
StoreKey         0
ProductKey       0
Quantity         0
Currency_Code    0
dtype: int64
Stores missing values:
 StoreKey         0
Country          0
State            0
Square_Meters    0
Open_Date        0
dtype: int64

Data types after processing:
Customers:
 CustomerKey             int64
Gender                 object
City                   object
State                  object
Country                object
Continent              object
Birthday       datetime64[ns]
dty